In [1]:
import sifce
import pandas as pd
import numpy as np
from bilby.core.prior import Uniform, Sine, Cosine, PowerLaw, Constraint, DeltaFunction
from pycbc.detector import Detector
from bilby.core.prior import PriorDict
from pycbc.psd import analytical

In [2]:
# get ifos
ifos=['H1', 'L1', 'V1']
ifos_det = []

for ifo in ifos:
    ifos_det.append(Detector(ifo))

In [3]:
prior_gw150914 = dict(
                    mass_1= PowerLaw(alpha=-1, name='mass_1', minimum=10, maximum=80),
                    mass_2= PowerLaw(alpha=-1, name='mass_2', minimum=10, maximum=80),
                    a_1 = Uniform(name='a_1', minimum=0, maximum=0.99),
                    a_2 = Uniform(name='a_2', minimum=0, maximum=0.99),
                    tilt_1 = Sine(name='tilt_1'),
                    tilt_2 = Sine(name='tilt_2'),
                    phi_12 = Uniform(name='phi_12', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                    phi_jl = Uniform(name='phi_jl', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                    theta_jn =  Sine(name='theta_jn'),
                    phase =  Uniform(name='phase', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                    reference_frequency = DeltaFunction(20),
                    dec=Cosine(name='dec'),
                    ra=Uniform(name='ra', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                    psi =  Uniform(name='psi', minimum=0, maximum=np.pi, boundary='periodic'),)

In [4]:
from sifce import population
from sifce import datatools

In [5]:
prior = PriorDict(dictionary = prior_gw150914)

pop = population.SimulationSet(distribution=prior)
pop.sample_distribution(1000)
# print(pop.simulations_dataframe)

In [6]:
sample_rate=1024
flow = 20.0
delta_f = 1.0/8
flen = int(sample_rate/delta_f)
psd = analytical.aLIGOZeroDetHighPower(flen, delta_f, flow)

psd_dict = {'H1': psd, 'L1': psd, 'V1':psd}
apx='IMRPhenomXPHM'

In [7]:
import time

start_time = time.time()

temp_pd = pop.simulations_dataframe
opt_snr_lst = list()

for index, row in temp_pd.iterrows():
    tmp_dict = row.to_dict()
    
    # get hp and hc:
    param_args = { k: tmp_dict[k] for k in pop.waveform_generation_labels}
    strain = dict()
    strain['plus'], strain['cross'] = datatools.compute_hphc_fd(param_args, apx)
    
    # projection into antenna:
    position_args = { k: tmp_dict[k] for k in pop.detector_position_labels}
    projected_strains = dict()
    for ifo_name, ifo_detec in zip(ifos, ifos_det):
        projected_strains[ifo_name] = dict(template = datatools.project_and_combine(position_args, strain, 0, detector_obj=ifo_detec))

    # calculate snr
    opt_snr_dict, temp_snr_dict = datatools.compute_snr_fd(projected_strains, psd_dict)
    opt_snr_lst.append(opt_snr_dict['net'])

pop.simulations_dataframe['opt_snr'] = opt_snr_lst
end_time = time.time()

In [8]:
print(end_time - start_time)

100.48945093154907


In [9]:
pop.simulations_dataframe

,mass_1,mass_2,spin_1x,spin_1y,spin_1z,spin_2x,spin_2y,spin_2z,theta_jn,phase,...,tilt_2,phi_12,phi_jl,reference_frequency,cal_idx,opt_snr,template_snr,iota,phi_1,phi_2
0,10.936731,48.217113,0.226618,0.145337,-0.395997,-0.538025,0.172325,-0.483351,1.575291,0.552719,...,2.278513,2.261370,0.342447,20.0,NaN,9736.918025,NaN,1.309144,0.570256,2.831625
1,72.445522,28.469030,-0.331859,-0.343401,-0.370218,0.414931,0.445626,-0.105908,2.064615,3.641671,...,1.743009,3.160156,4.505344,20.0,NaN,19567.517103,NaN,2.404853,3.944082,0.821053
2,16.933450,52.381419,-0.153855,0.381383,0.310845,0.217495,0.861578,0.331442,1.404174,5.903639,...,1.213789,5.652468,4.125985,20.0,NaN,17687.346389,NaN,1.827052,1.954242,1.323525
3,11.343661,33.264689,-0.252132,0.121526,-0.708990,-0.139240,-0.174189,-0.388386,0.257130,4.641940,...,2.620380,1.345582,5.931607,20.0,NaN,21439.565770,NaN,0.384931,2.692455,4.038037
4,39.293469,29.710642,-0.097050,0.368731,-0.157010,-0.256704,0.077837,-0.199495,1.435245,4.135194,...,2.210254,1.019028,3.073251,20.0,NaN,20763.559185,NaN,1.425604,1.828159,2.847187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,57.797293,20.073455,0.012743,-0.094358,-0.166677,-0.064576,-0.251338,0.147642,0.681529,3.791886,...,1.053525,5.897458,5.480723,20.0,NaN,37866.011244,NaN,0.758422,4.846626,4.460898
996,51.327213,54.264326,-0.015432,0.035845,-0.333862,-0.156883,-0.177034,0.085683,1.106055,4.717150,...,1.223270,2.009922,5.442758,20.0,NaN,32082.832615,NaN,1.158421,1.977343,3.987265
997,27.299733,14.228586,-0.132832,0.141570,0.157146,0.173655,0.086936,0.087678,0.892951,5.494103,...,1.146709,4.422974,4.384820,20.0,NaN,23976.766074,NaN,0.960958,2.324360,0.464149
998,15.284652,66.877541,0.088692,0.545079,-0.097324,-0.015110,0.117628,-0.041295,1.768299,5.778240,...,1.905871,0.289057,4.299389,20.0,NaN,6461.287868,NaN,1.927017,1.409495,1.698552
